In [1]:
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd
import shutil
import numpy as np
import seaborn as sns
import IPython.display as ipd
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
#import soundfile as sf
import os
import numpy as np
from PIL import Image
from scipy.fftpack import fft
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
%matplotlib inline


Using TensorFlow backend.


In [ ]:
train_path = './train/audio/'
classes = os.listdir(train_path)

In [ ]:
def class_counts(path, classes):
    total_count = 0
    d = {}
    for c in classes:
        i = len(os.listdir(path + c))
        print(c, ':', i)
        d[c] = i
        total_count += i
    print('\n')
    print(f'total counts: {total_count}')
    return d

In [ ]:
full_train_path = './train/audio/'
class_dict = class_counts(full_train_path, classes)

In [ ]:
train_audio_path = "./train/audio"
train_labels = os.listdir(train_audio_path)
print(f'Number of labels: {len(train_labels)}')

wavs = []
labels = []
for label in train_labels:
    if label == '_background_noise_':
        continue
    files = os.listdir(train_audio_path + '/' + label)
    for f in files:
        if not f.endswith('wav'):
            continue
        wavs.append(f)
        labels.append(label)

train = pd.DataFrame({'file':wavs,'label':labels})
train.info()
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(ax=ax, x="label", data=train)
print(train.label.unique())

In [ ]:
duration_of_recordings=[]
for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + '/' + label + '/' + wav)
        duration_of_recordings.append(float(len(samples)/sample_rate))
    
plt.hist(np.array(duration_of_recordings))

In [ ]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', '_background_noise_']

In [ ]:
all_wave = []
all_label = []
for label in classes:
    print(label)
    waves = [f for f in os.listdir(train_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

In [ ]:
os.makedirs(train_path+'unknown', exist_ok=True)

In [ ]:
def moveToUnknown(path):
    dirs = os.listdir(path)
    for c in dirs:
        # moving if not in labels list
        if c not in labels:
            fnames = os.listdir(path + c)
            for fname in fnames:
                src = path + c + '/' + fname
                dst = path + 'unknown' + '/' + c + '_' + fname
                shutil.move(src, dst)

In [ ]:
moveToUnknown(train_path)
train_counts = class_counts(train_path, classes + ['unknown']) 

In [ ]:
os.makedirs(train_path+'silence', exist_ok=True)

In [ ]:
background_path = './train/audio/_background_noise_/'

In [ ]:
background_fnames = [f for f in os.listdir(background_path) if 'wav' in f]

In [ ]:
sample_rate = 8000
n_train = 500

for back_fname in background_fnames:
    sample_rate, samples = wavfile.read(background_path + back_fname)
    train_background_arr = samples[:int(len(samples)*0.8)]
    train_len = train_background_arr.shape[0]
    
    #create clips for training n_train times
    for i in range(n_train):
        start_idx = np.random.choice(range(train_len - sample_rate))
        seq_idx = range(start_idx, start_idx+sample_rate)
        new_clip = train_background_arr[seq_idx]
        wavfile.write('./train/audio/silence/' + f'_{i}' + back_fname  ,sample_rate, new_clip)

In [ ]:
train_path = './train/audio/'
classes = os.listdir(train_path)
train_counts=class_counts(train_path, classes)

In [18]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go','silence','unknown']
dirs = os.listdir(train_path)
for c in dirs:
    if c not in labels:
        shutil.rmtree(train_path + c)


NameError: name 'train_path' is not defined

In [ ]:
audio_path = './train/audio/'
pict_Path = './picts/train/audio/'
test_pict_Path = './picts/test/audio'
test_audio_path = './test/audio'
samples = []

In [ ]:
subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)

In [ ]:
if not os.path.exists(pict_Path):
    os.makedirs(pict_Path)

if not os.path.exists(test_pict_Path):
    os.makedirs(test_pict_Path)


subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)
        if not os.path.exists(pict_Path + '/' + x):
            os.makedirs(pict_Path +'/'+ x)

In [ ]:
sample_audio = []
total = 0
for x in subFolderList:
    
    # get all the wave files
    all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
    total += len(all_files)
    #collect the first file from each dir
    #print(all_files)
    sample_audio.append(audio_path  + x + '/'+ all_files[0])
    
    # show file counts
    print('count: %d : %s' % (len(all_files), x ))
print(total)

In [ ]:
sample_audio[0]

In [ ]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

In [ ]:
fig = plt.figure(figsize=(10,10))

# for each of the samples
for i, filepath in enumerate(sample_audio[:9]):
    # Make subplots
    plt.subplot(3,3,i+1)
    
    # pull the labels
    label = filepath.split('/')[-2]
    plt.title(label)
    
    # create spectogram
    samplerate, test_sound  = wavfile.read(filepath)
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.axis('off')

In [ ]:
five_samples = [audio_path + 'yes/' + y for y in os.listdir(audio_path + 'yes/')[:6]]

fig = plt.figure(figsize=(10,10))

for i, filepath in enumerate(five_samples):
    # Make subplots
    plt.subplot(3,3,i+1)
    
    # pull the labels
    label = filepath.split('/')[-1]
    plt.title('"yes": '+label)
    
    # create spectogram
    # create spectogram
    samplerate, test_sound  = wavfile.read(filepath)
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.axis('off')


In [ ]:
fig = plt.figure(figsize=(3,3))
for i, filepath in enumerate(sample_audio[:6]):
    plt.subplot(9,1,i+1)
    samplerate, test_sound  = wavfile.read(filepath)
    plt.title(filepath.split('/')[-2])
    plt.axis('off')
    plt.plot(test_sound)

In [ ]:
fig = plt.figure(figsize=(8,20))
for i, filepath in enumerate(five_samples):
    plt.subplot(9,1,i+1)
    samplerate, test_sound = wavfile.read(filepath)
    plt.title(filepath.split('/')[-2])
    plt.axis('off')
    plt.plot(test_sound)

In [ ]:
def wav2img(wav_path, targetdir= '', figsize=(3,3)):
    """
    takes in wave file path
    and the fig size. Default 4,4 will make images 288 x 288
    """

    fig = plt.figure(figsize=figsize)    
    # use soundfile library to read in the wave files
    samplerate, test_sound  = wavfile.read(filepath)
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    ## create output path
    output_file = wav_path.split('/')[-1].split('.wav')[0]
    output_file = targetdir +'/'+ output_file
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.imsave('%s.png' % output_file, spectrogram)
    plt.close()

In [ ]:
for i, x in enumerate(subFolderList[:]):
    print(i, ':', x)
    # get all the wave files
    
    all_files = [y for y in os.listdir(audio_path + x) if '.wav' in y]
    for file in all_files[:]:
        wav2img(audio_path + x + '/' + file, pict_Path + x)

In [ ]:
all_files = [y for y in os.listdir(test_audio_path) if '.wav' in y]
for file in all_files:
    wav2img(test_audio_path+file , test_pict_Path)

In [ ]:
data_dir_list = os.listdir(pict_Path)
train_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(pict_Path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = pict_Path + '/'+ dataset + '/'+ img
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        train_list.append(x)

In [2]:
model = VGG19(weights = "imagenet", include_top=True, input_shape = (224, 224, 3))
model.summary()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of cbe5617147190e668d6c5d5026f83318 so we will re-download the data.
574717952/574710816 [==============================] - 655s 1us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856

In [6]:
last_layer = model.get_layer('fc2').output
out = Dense(12, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(input=model.input, output = out)
custom_vgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

C:\Users\sahithi katuri\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ou...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
opt = Adam(0.001)
custom_vgg_model.compile( optimizer = opt, loss = "categorical_crossentropy", metrics=["accuracy"])

In [8]:
data = ImageDataGenerator(rescale=(1./255),validation_split=0.2)
# prepare an iterators for  dataset
train_set = data.flow_from_directory('./picts/train/audio',
subset='training',
target_size = (224, 224),
batch_size = 32,
class_mode = 'categorical')
X_train,y_train  = next(train_set)
test_set = data.flow_from_directory('./picts/train/audio',
subset='validation',
target_size = (224, 224),
batch_size = 32,
class_mode = 'categorical')
X_test, y_test= next(test_set)

Found 7742 images belonging to 12 classes.
Found 1928 images belonging to 12 classes.


In [10]:
epochs = 10
batch_size = 32
custom_vgg_model.fit_generator(train_set,
                   epochs=epochs,
                   steps_per_epoch=10,
                   validation_steps=30,
                   validation_data=test_set,
                   verbose=1)        

Epoch 1/10
10/10 [==============================] - 984s 98s/step - loss: 128.7907 - accuracy: 0.1750 - val_loss: 12.1638 - val_accuracy: 0.3167
Epoch 2/10
10/10 [==============================] - 1041s 104s/step - loss: 3.7453 - accuracy: 0.3844 - val_loss: 1.6421 - val_accuracy: 0.3073
Epoch 3/10
10/10 [==============================] - 999s 100s/step - loss: 2.0264 - accuracy: 0.4156 - val_loss: 1.9215 - val_accuracy: 0.4231
Epoch 4/10
10/10 [==============================] - 1044s 104s/step - loss: 1.8932 - accuracy: 0.3781 - val_loss: 1.8103 - val_accuracy: 0.2990
Epoch 5/10
10/10 [==============================] - 1117s 112s/step - loss: 1.9578 - accuracy: 0.3938 - val_loss: 1.9961 - val_accuracy: 0.3024
Epoch 6/10
10/10 [==============================] - 962s 96s/step - loss: 1.6979 - accuracy: 0.4750 - val_loss: 1.5637 - val_accuracy: 0.3208
Epoch 7/10
10/10 [==============================] - 1010s 101s/step - loss: 1.8048 - accuracy: 0.3344 - val_loss: 1.4291 - val_accuracy: 0

In [11]:
score = custom_vgg_model.evaluate(X_test,y_test, batch_size=10,verbose=1)
print('Accuracy is calculated for test')
print('Test loss:',score[0])
print('Test Accuracy:',score[1])
print('Accuracy is calculated for train')
score=custom_vgg_model.evaluate(X_train,y_train,batch_size=10,verbose=1)
print('Train accuracy:',score[1])
custom_vgg_model.summary()


32/32 [==============================] - 17s 529ms/step
Accuracy is calculated for test
Test loss: 1.6536725610494614
Test Accuracy: 0.46875
Accuracy is calculated for train
32/32 [==============================] - 16s 510ms/step
Train accuracy: 0.375
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_____

In [13]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [26]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go','silence','unknown']
import IPython.display as ipd
import random
index=random.randint(0,len(X_test)-1)
print(index)
samples=X_test[index]
print("Audio:",labels[np.argmax(y_test[index])])


12
Audio: off


ValueError: Array audio input must be a 1D or 2D array

In [35]:
sample = X_test[0]
y_pred = custom_vgg_model.predict_generator(test_set,steps=1)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


In [36]:
y_true = test_set.classes
print(y_true)

[ 0  0  0 ... 11 11 11]


In [38]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))


print('Classification Report')
print(classification_report(y_true, y_pred))

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [1928, 32]

In [ ]:
score = custom_vgg_model.evaluate(X_test,y_test, batch_size=10,verbose=1)
print('Accuracy is calculated for test')
print('Test loss:',score[0])
print('Test Accuracy:',score[1])
print('Accuracy is calculated for train')
score=custom_vgg_model.evaluate(X_train,y_train,batch_size=10,verbose=1)
print('Train accuracy:',score[1])
custom_vgg_model.summary()

In [ ]:
y_pred1 =  np.argmax(Y_pred, axis=1)
y_test_ohe_decoded  = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test_ohe_decoded,y_pred1 )
print("Confusion Matrix")
print(cm)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
precision = precision_score(y_test_ohe_decoded,y_pred1, average=None)
recall = recall_score(y_test_ohe_decoded,y_pred1, average=None)
f1 = f1_score(y_test_ohe_decoded,y_pred1 , average=None)
print('Precision for EACH class = ', precision)
print("Recall for EACH class = ", recall)
print("F1 Score for EACH class = ",f1)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
#ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
  fpr[i] , tpr[i] , _ = roc_curve(y_test[:,i], Y_pred[:,i])
  roc_auc[i] = auc(fpr[i],tpr[i])
# Plot of a ROC curve for a specific class
for i in range(3):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")1
    plt.show()